In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io

In [ ]:
csv_file_name = "2025-01.csv"
df_raw = pd.DataFrame()
df_raw = pd.read_csv(csv_file_name)
print("Tamaño del DataFrame:")
print(df_raw.shape)

print("\nPrevisualización del dataFrame: ")
display(df_raw.head(50))

## Fase 2: Transformación

In [ ]:
print("\n--- FASE 2: TRANSFORMACIÓN ---")

# Hacemos una copia para no modificar el DataFrame original
df = df_raw.copy()

# --- 2.1 Limpieza de Datos (Cleaning) ---
print("\nIniciando limpieza de datos...")

# Mostrar tipos de datos antes de la transformación
print("\nTipos de datos antes de la conversión a datetime:")
print(df[['Fecha_Retiro', 'Fecha_Arribo']].dtypes)

# Convertir columnas de fecha a formato datetime
df['Fecha_Arribo'] = pd.to_datetime(df['Fecha_Arribo'], dayfirst=True, errors='coerce')
df['Fecha_Retiro'] = pd.to_datetime(df['Fecha_Retiro'], dayfirst=True, errors='coerce')
print("\nColumnas de fecha convertidas a datetime.")

# Mostrar tipos de datos despues de la transformación
print("\nTipos de datos despues de la conversión a datetime:")
print(df[['Fecha_Retiro', 'Fecha_Arribo']].dtypes)

# Mostrar una previsualización del DataFrame después de la transformación
print("\nPrevisualización del DataFrame después de la conversión:")
display(df.head())











### Checar valores nulos en columnas clave

In [ ]:
# Manejo de Valores Nulos (Imputación)
print(f"Valores nulos en 'Ciclo_Estacion_Retiro': {df['Ciclo_Estacion_Retiro'].isnull().sum()}")
print(f"Valores nulos en 'Ciclo_EstacionArribo': {df['Ciclo_EstacionArribo'].isnull().sum()}")
print(f"Valores nulos en 'Bici': {df['Bici'].isnull().sum()}")
print(f"Valores nulos en 'Fecha_Retiro': {df['Fecha_Retiro'].isnull().sum()}")
print(f"Valores nulos en 'Hora_Retiro': {df['Hora_Retiro'].isnull().sum()}")

In [ ]:
print("\nConteo de valores nulos por columna:")
print(df.isnull().sum())

## Visualización de variable género

In [ ]:
# Visualizar la distribución de la variable categórica 'genero'
print("\nVisualización de la distribución de 'genero':")

# Imprimir la cantidad de cada variable categórica en 'genero'
print("\nConteo de viajes por género:")
print(df['Genero_Usuario'].value_counts())

plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Genero_Usuario', palette='viridis', hue='Genero_Usuario', legend=False)
plt.title('Distribución de Género')
plt.xlabel('Género')
plt.ylabel('Cantidad de Viajes')
plt.show()

### Manejo de valores Other en género

In [ ]:
df['Genero_Usuario'] = df['Genero_Usuario'].fillna("OTHER")
df['Genero_Usuario'] = df['Genero_Usuario'].replace(['?', 'O'], 'OTHER')
df['Genero_Usuario'] = df['Genero_Usuario'].replace(['Otros'], 'OTHER')
print("Valores nulos en la columna 'genero' imputados con 'OTHER'.")

# Verificar si aún quedan valores nulos en la columna 'genero'
print(f"Valores nulos restantes en 'genero': {df['Genero_Usuario'].isnull().sum()}")

In [ ]:
print("\nVisualización de la distribución de 'genero':")

# Imprimir la cantidad de cada variable categórica en 'genero'
print("\nConteo de viajes por género:")
print(df['Genero_Usuario'].value_counts())

plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Genero_Usuario', palette='viridis', hue='Genero_Usuario', legend=False)
plt.title('Distribución de Género')
plt.xlabel('Género')
plt.ylabel('Cantidad de Viajes')
plt.show()

In [ ]:
print("\nConteo de viajes por género:")
print(df['Genero_Usuario'].value_counts())

## Uso de One-Hot Encoding

In [ ]:
# Aplicar One-Hot Encoding a las columnas 'genero' y 'modelo_bicicleta'
df_encoded = pd.get_dummies(df, columns=['Genero_Usuario'], prefix=['Genero'])

# Mostrar las primeras filas del DataFrame codificado
print("\nDataFrame después de One-Hot Encoding:")
display(df_encoded.head())

### Discretización

En este caso, no existe una columna per se que se pueda discretizar. Tendríamos que crear una nueva. 

In [ ]:
'''
print("\nIniciando discretización de 'duracion_recorrido'...")

# Definir los rangos y las etiquetas
bins = [0, 1200, 2400, 3600, df['duracion_recorrido'].max()]
labels = ['Viaje Corto', 'Viaje Mediano', 'Viaje Largo', 'Viaje Muy Largo (Anomalía)']

# Aplicar la discretización
df_encoded['categoria_duracion'] = pd.cut(df_encoded['duracion_recorrido'], bins=bins, labels=labels, right=False, include_lowest=True)

print("Columna 'categoria_duracion' creada.")

# Mostrar el conteo de valores en la nueva columna
print("\nConteo de viajes por categoría de duración:")
print(df_encoded['categoria_duracion'].value_counts())

# Mostrar las primeras filas con la nueva columna
print("\nPrevisualización del DataFrame con la nueva columna:")
display(df_encoded.head())
'''

### Normalización

In [ ]:
# Aplicar Normalización (Min-Max Scaling) a las columnas numéricas
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

columns_to_normalize = [
    "Edad_Usuario"
]

df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

print("\nColumnas numéricas normalizadas:")
display(df_encoded.head())

En este caso, es más claro ver la edad sin normalización

In [ ]:
df_encoded[columns_to_normalize] = scaler.inverse_transform(df_encoded[columns_to_normalize])

print("\nColumnas numéricas después de deshacer la normalización:")
display(df_encoded.head())

In [ ]:
# --- Verificación del DataFrame Transformado ---
print("\n--- Vista previa del DataFrame transformado ---")
display(df_encoded.head(20))

### Feature Engineering

In [ ]:
# --- Feature Engineering ---
print("\nIniciando Feature Engineering...")
# Crear nuevas características a partir de las existentes.

# 1. Duración del viaje en minutos
df_encoded['duracion_minutos'] = (df_encoded['Fecha_Arribo'] - df_encoded['Fecha_Retiro']).dt.total_seconds() / 60

# 2. Día de la semana (0=Lunes, 6=Domingo)
df_encoded['dia_semana'] = df_encoded['Fecha_Retiro'].dt.dayofweek

# 2.1 Día de la semana (0=Lunes, 6=Domingo)
df_encoded['dia_semana_arribo'] = df_encoded['Fecha_Arribo'].dt.dayofweek

# 3. Hora del día
df_encoded['hora_inicio'] = df_encoded['Fecha_Retiro'].dt.hour

# 3.1. Hora del día
df_encoded['hora_final'] = df_encoded['Fecha_Arribo'].dt.hour

# 4. Categoría de día (Fin de semana vs. Entre semana)
df_encoded['tipo_dia'] = df_encoded['dia_semana'].apply(lambda x: 'Fin de Semana' if x >= 5 else 'Entre Semana')

In [ ]:
print("Nuevas características creadas: 'duracion_minutos', 'dia_semana', 'hora_inicio', 'tipo_dia'.")
display(df_encoded.head())

### Distancia del viaje

En este caso, no contamos con los datos para sacar la distancia

In [ ]:
from geopy.distance import geodesic
'''
def calculate_distance(row):
    origin_coords = (row['lat_estacion_origen'], row['long_estacion_origen'])
    destination_coords = (row['lat_estacion_destino'], row['long_estacion_destino'])
    try:
        return geodesic(origin_coords, destination_coords).km
    except ValueError:
        return np.nan # Handle potential errors with coordinates

df_encoded['distancia_viaje'] = df_encoded.apply(calculate_distance, axis=1)

print("Nuevas características creadas: 'duracion_minutos', 'dia_semana', 'hora_inicio', 'tipo_dia', 'distancia_viaje'.")
'''

## Fase 3: Loading

In [ ]:
# =============================================================================
# FASE 3: CARGA (Load)
# =============================================================================
print("\n--- FASE 3: CARGA ---")

if not df_encoded.empty:
    # El paso final es cargar nuestros datos limpios y enriquecidos a un nuevo destino.
    # Opción 1: Guardar como un nuevo archivo CSV.
    try:
        df_encoded.to_csv("ecobici_limpios.csv", index=False)
        print("DataFrame transformado guardado exitosamente en 'ecobici_limpios.csv'")
    except Exception as e:
        print(f"Error al guardar el archivo CSV: {e}")

    # Opción 2 (Recomendada para Big Data): Guardar en formato Parquet.
    # Parquet es un formato columnar optimizado para análisis, es más rápido y ocupa menos espacio.
    try:
        # Necesitarás instalar 'pyarrow' o 'fastparquet': pip install pyarrow
        df_encoded.to_parquet("ecobici_limpios.parquet", index=False)
        print("DataFrame transformado guardado exitosamente en 'ecobici_limpios.parquet'")
    except ImportError:
        print("\nPara guardar en formato Parquet, necesitas instalar 'pyarrow'.")
        print("Ejecuta: pip install pyarrow")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet: {e}")